# SANA-FE Outputs and Traces #

Now that you have seen how to define hardware architectures and SNNs,
we will look at the outputs SANA-FE can generate. Note that this tutorial uses the Python interface -
if you want to see how to generate traces running from the command line see
the `README`


The first useful output is one you should have already seen: the run summary. This
is printed, saved as a YAML file `run_summary.yaml`, or returned as a Python
Dictionary. We will run our example SNN running on the example architecture
from earlier.


In [ ]:
# TODO HACK deleteme
import sys
import os
PROJECT_DIR = "/home/usr1/jboyle/neuro/sana-fe"
sys.path.insert(0, os.path.join(PROJECT_DIR))

In [ ]:
#%pip install --extra-index-url https://test.pypi.org/simple/ sanafe==0.0.3
%pip install pyyaml
!wget -nc https://raw.githubusercontent.com/SLAM-Lab/SANA-FE/cpp/tutorial/arch.yaml
!wget -nc https://raw.githubusercontent.com/SLAM-Lab/SANA-FE/cpp/tutorial/snn.yaml
import sanafe
import yaml

In [ ]:
# Load the toy architecture and SNN again, similar to the one used in the last
#  few tutorials
arch = sanafe.load_arch("arch.yaml")
snn = sanafe.Network()
snn.create_neuron_group("in", 2)
snn.create_neuron_group("out", 2)

snn.groups["in"].neurons[0].configure(log_spikes=True, log_potential=True,
                                      model_parameters={"bias": 0.2, "threshold": 1.0})
snn.groups["in"].neurons[1].configure(log_spikes=True, log_potential=True,
                                      model_parameters={"bias": 0.5, "threshold": 1.0})
snn.groups["out"].neurons[0].configure(model_parameters={"threshold": 2.0})
snn.groups["out"].neurons[1].configure(model_parameters={"threshold": 2.0})

snn.groups["in"].neurons[0].connect_to_neuron(snn.groups["out"].neurons[0], {"weight": -1.0})
snn.groups["in"].neurons[0].connect_to_neuron(snn.groups["out"].neurons[1], {"weight": -2.0})
snn.groups["in"].neurons[1].connect_to_neuron(snn.groups["out"].neurons[1], {"weight": 3.0})

snn.groups["in"].neurons[0].map_to_core(arch.tiles[0].cores[0])
snn.groups["in"].neurons[1].map_to_core(arch.tiles[0].cores[0])
snn.groups["out"].neurons[0].map_to_core(arch.tiles[0].cores[0])
snn.groups["out"].neurons[1].map_to_core(arch.tiles[0].cores[0])
print(snn)

chip = sanafe.SpikingChip(arch, record_spikes=True, record_potentials=True)
chip.load(snn)
# This time we only need to run for 10 timesteps
results = chip.sim(10, heartbeat=1)


In [ ]:
# Pretty-print the run summary, which is always formatted as YAML
print(yaml.dump(results))


## SNN Traces ##

It is possible to extract more detail from the simulation via SANA-FE's various
trace formats. First we will focus on the SNN traces. These
give us insight in the dynamics of the SNN being executed.

SANA-FE supports both spike and neuron potential traces. Spike
traces tell us when each neuron fired and sent a spike, commonly visualized as
'raster' plots in neuroscience. SANA-FE will save a file `spikes.csv` with a
line per spike.

SNN traces are enabled/disabled globally, but also filtered by per-neuron
spike and voltage probes. If spike traces are enabled, the simulator will record
spikes for probed neurons, i.e., any neurons with attribute `log_spikes` set.
SANA-FE will output a CSV file with the compressed format:

   `<neuron>,<timestep>`



In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import csv

## Create the spike raster plot
timesteps = []
spikes = []
print("Spike data:")
with open("spikes.csv") as spikes_file:
    spike_reader = csv.DictReader(spikes_file)
    for line in spike_reader:
        print(line)
        (nid, timestep) = line["neuron"], line["timestep"]
        group, neuron = nid.split(".")
        group, neuron, timestep = group, int(neuron), int(timestep)
        timesteps.append(timestep)
        spikes.append(neuron)

fig = plt.figure(figsize=(5, 2))
ax = plt.gca()
colors = matplotlib.colors.ListedColormap(("#ff7f0e", "#1f77b4"))
ax.scatter(timesteps, spikes, marker="|", s=700, linewidths=3, c=spikes,
           cmap=colors)
ax.set_xlabel("Time-step")
ax.set_xlim((0, 10.1))
ax.set_xticks((0, 2, 4, 6, 8, 10))

plt.minorticks_on()
ax.yaxis.set_tick_params(which='minor', bottom=False)
ax.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(1))

ax.set_ylim((-0.5, 1.5))
ax.set_yticks((0, 1))
_ = ax.set_yticklabels(("Neuron in.0", "Neuron in.1"))

Similarly, potential traces can be enabled globally and filtered locally to record the
voltages at probed neurons. Voltage probes are set via the attribute `log_potential`.
The potentials are recorded in another CSV file using one column per probe and one line per
time-step.


In [ ]:
import numpy as np

# Extract the voltages from the CSV file
voltages = np.zeros((11, 2))
with open("potentials.csv", "r") as v_file:
    v_reader = csv.DictReader(v_file)
    for line in v_reader:
        print(line)
        (timestep, v1, v2) = (int(line["timestep"]), float(line["neuron in.0"]),
                              float(line["neuron in.1"]))
        voltages[timestep, 0] = v1
        voltages[timestep, 1] = v2

plt.figure(figsize=(6, 2.5))
ax = plt.gca()
ax.plot(voltages[:, 1], '--^')
ax.plot(voltages[:, 0], '-o')
ax.set_ylabel("Neuron Potential (V)")
plt.legend(("Neuron 0.1", "Neuron 0.0"))

ax.yaxis.set_tick_params(which='minor', bottom=False)
ax.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(1))
ax.set_xlim((0, 10.1))
_ = ax.set_xlabel("Time-step")

## Hardware Traces ##

In addition to information on SNN behavior, SANA-FE can also output detailed
per-timestep performance data when requested. SANA-FE has two types of hardware
trace: hardware *performance* traces and spike *message* traces.

Hardware performance traces give much more insight into the breakdown of on-chip
activity over time. In this toy example, we plotted how many neurons fired on each
time-step and the total energy consumed by the chip per time-step.

In [ ]:
# Run again but with hardware traces enabled
chip = sanafe.SpikingChip(arch, record_perf=True, record_messages=True)
chip.load(snn)
results = chip.sim(10)

In [ ]:
fig, ax = plt.subplots(2, figsize=(5, 4))
fired = np.zeros((11))
energy = np.zeros((11))
with open("perf.csv", "r") as perf_file:
    reader = csv.DictReader(perf_file)
    for line in reader:
        print(line)
        (timestep, f, e) = (int(line["timestep"]), int(line["fired"]),
                              float(line["total_energy"]))
        fired[timestep] = f
        energy[timestep] = e

ax[0].plot(fired, '-o')
ax[0].set_ylabel("Neurons Fired")
ax[1].plot(energy * 1.0e12, '-o')
ax[1].set_ylabel("Total Energy (pJ)")

plt.minorticks_on()
ax[1].xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(1))

ax[0].set_xlim((0, 10.1))
ax[1].set_xlim((0, 10.1))
ax[0].set_xticks((0, 2, 4, 6, 8, 10))
ax[1].set_xticks((0, 2, 4, 6, 8, 10))
_ = ax[1].set_xlabel("Time-step")


Finally, the messages trace `messages.csv` contains information about all packets sent to the
network during the simulation, and includes information about each packet like
its various delays and how many hops it makes. This trace is useful if you
are interested in the network performance, or want to simulate the network in
even more detail e.g., using an event-drive or cycle-accurate
simulator. This information is more niche, and so we don't visualize it in
this tutorial.